In [1]:
import Management.medOOP_3D
from Visualisation import medVIZ_3D
import Parsing.parse_funcs as pf



ModuleNotFoundError: No module named 'Conversion_utilities'

In [ ]:
dicom_root_dir = "/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom"
nifti_output_root_dir = "/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Nifti"

dir_list = get_sub_directories(dicom_root_dir)
dir_list = inner_sub_dir_replace(dir_list) 
paired_list = get_path_id_list(dir_list)


In [ ]:
paired_list

[('/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom/Subject (1009)',
  'Subject_(1009)'),
 ('/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom/Subject (1003)',
  'Subject_(1003)'),
 ('/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom/Subject (1000)',
  'Subject_(1000)'),
 ('/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom/Subject (1011)',
  'Subject_(1011)'),
 ('/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom/Subject (1018)',
  'Subject_(1018)'),
 ('/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom/Subject (1012)',
  'Subject_(1012)'),
 ('/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom/Subject (102)/837967-',
  'Subject_(102)'),
 ('/Users/eamonmcandrew/Desktop/ML/CT_work/Covid_Positive_CT/Covid_Positive_CT_Dicom/Subject (1016)',
  'Subject_(1016)'),
 ('/Users/

In [ ]:

Paitent_1 = OOP_3D.Paitent(paired_list[0][0])

In [ ]:
# Paitent_1.LUNG.display_3D_volume()

In [ ]:
def dash_ct_view(Scan):

    name = Scan.getName()
    fig = Scan.display_3D_volume()
    
    import plotly.graph_objects as go # or plotly.express as px # or any Plotly Express function e.g. px.bar(...)
    # fig.add_trace( ... )
    fig.update_layout(title=go.layout.Title(text=name))

    import dash
    import dash_core_components as dcc
    import dash_html_components as html

    app = dash.Dash()
    app.layout = html.Div([
    dcc.Graph(figure=fig)
    ])

    return app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter


In [ ]:
# dash_ct_view(Paitent_1.LUNG)

In [ ]:
Dataset_1 = OOP_3D.Dataset(paired_list)


In [ ]:
Dataset_1.num_paitents

30